# 🧠 EEG Brain State Classifier — Hackathon Demo

## Can a computer tell if you're relaxed or stressed... just from your brainwaves?

**What we built:** A machine learning system that:
1. Reads brainwave data from an OpenBCI EEG headset (8 electrodes on your head)
2. Figures out if you are **Relaxed** or **Stressed / Focused**
3. When you're stressed → it **automatically plays soothing music** to help you relax!

---

### 🗺️ Notebook Roadmap (What We'll Do)

| Step | What Happens | Why It Matters |
|------|-------------|----------------|
| **Part A** | Install tools & load data | Get our brainwave files ready |
| **Part B** | Build a SIMPLE model (the "bad" one) | See why naive approaches fail |
| **Part C** | Fix the problems & build a BETTER model | Learn how real data science works |
| **Part D** | Simulated live demo with music | Show it working on a recording |
| **Part E** | REAL live demo with headset | The showstopper! |
| **Part F** | How to answer judge questions | Be ready for anything |

---

### 🧪 The Science Behind It

Your brain produces tiny electrical signals called **brainwaves**. Different mental states produce different patterns:

```
  😴 Delta waves (1-4 Hz)   → Deep sleep
  🧘 Theta waves (4-8 Hz)   → Meditation, daydreaming
  😌 Alpha waves (8-13 Hz)  → Relaxed, calm, eyes closed
  🤔 Beta waves  (13-30 Hz) → Active thinking, problem-solving
  🧠 Gamma waves (30-45 Hz) → Intense focus, learning
```

**Key insight:** When you're relaxed, your brain produces MORE alpha waves. When you're stressed or thinking hard, beta waves go up and alpha goes down.

We measure the STRENGTH (power) of each wave type and let a machine learning model learn the pattern!


---
#  PART A: Setup & Load Data

## Step A1: Install the tools we need

Think of these like apps on your phone — each one does something special:
- **pandas** = reads data files (like Excel for Python)
- **numpy** = does math on numbers
- **scipy** = signal processing (analyzing wave patterns)
- **scikit-learn** = machine learning (the "brain" of our classifier)
- **matplotlib** = draws charts and graphs
- **pygame** = plays music
- **brainflow** = talks to the OpenBCI headset


In [ ]:
# Run this cell to install everything (takes ~30 seconds)
!pip install pandas numpy scipy scikit-learn matplotlib pygame brainflow joblib -q
print('All tools installed!')

: 

## Step A2: Import the tools into our notebook
This is like opening the apps we just installed.

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.signal import welch
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import joblib
import warnings
import time
warnings.filterwarnings('ignore')

print("All libraries loaded!")


## Step A3: Define the brainwave frequency bands

This is where we tell the computer what frequency ranges correspond to which brain waves.

**Analogy:** Imagine a radio — you tune to different frequencies to hear different stations. Our brain has "stations" too!

In [ ]:
# Brainwave frequency bands (in Hertz = cycles per second)
BANDS = {
    'delta': (1, 4),      # Very slow waves - deep sleep
    'theta': (4, 8),      # Slow waves - meditation, drowsiness
    'alpha': (8, 13),     # Medium waves - RELAXED (the key one!)
    'beta':  (13, 30),    # Fast waves - THINKING HARD / STRESSED
    'gamma': (30, 45)     # Very fast waves - intense focus
}

SAMPLING_RATE = 250   # Our headset records 250 data points per second
NUM_CHANNELS = 8      # We have 8 electrodes on the head

print("Brainwave bands defined:")
for name, (low, high) in BANDS.items():
    bar = "#" * (high - low)
    print(f"   {name:6s} : {low:2d} - {high:2d} Hz  {bar}")


## Step A4: Connect to Google Drive and load our EEG data files

We recorded brainwave data using the OpenBCI headset while the subject was in different mental states. Each file is named like:
- `relaxation_01.csv` - person was relaxing
- `focus_02.csv` - person was doing math problems
- `negative_03.csv` - person was watching stressful videos
- `meditation_01.csv` - person was meditating

In [ ]:
# Connect to Google Drive (where our data lives)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    DATA_FOLDER = '/content/drive/MyDrive/eeg_data'
    IN_COLAB = True
    print("Google Drive connected!")
except ImportError:
    DATA_FOLDER = './eeg_data'
    IN_COLAB = False
    print("Running locally - using ./eeg_data folder")

# Show all the files we have
print(f"\nData folder: {DATA_FOLDER}")
print(f"\nFiles we found:")
print("-" * 60)

csv_files = []
for f in sorted(os.listdir(DATA_FOLDER)):
    if f.endswith('.csv') and 'test' not in f.lower():
        csv_files.append(f)
        raw_label = f.split('_')[0]
        print(f"   {f:35s}  (label: '{raw_label}')")

print("-" * 60)
print(f"\nTotal training files: {len(csv_files)}")


---
# 🅱️ PART B: The "Naive" Model (Low Accuracy — Let's See Why!)

Before we build the good model, let's see what happens when we just throw the data at machine learning without thinking carefully. **This is an important lesson: garbage in = garbage out!**

### What we'll do:
1. Load each file and extract ONE set of features per file
2. Use the raw filename as the label
3. Train a Random Forest and see how badly it does

### What is Power Spectral Density (PSD)?
Imagine you have a song playing. PSD is like an equalizer display that shows you which frequencies are present and how loud each one is. For brainwaves, PSD tells us how much power is in each frequency band. More alpha power = more relaxed!

The **Welch method** calculates PSD by:
1. Split the signal into overlapping chunks
2. Calculate the frequency content of each chunk (using Fourier Transform)
3. Average them together (this reduces noise and gives a cleaner picture)

In [ ]:
# SIMPLE (NAIVE) FEATURE EXTRACTION
# One feature vector per entire file

def extract_features_simple(filename):
    # Get label from filename: "relaxation_01.csv" -> "relaxation"
    basename = os.path.basename(filename)
    label = basename.split('_')[0]

    # Read the CSV file (tab-separated, no header row)
    df = pd.read_csv(filename, header=None, sep='\t')

    if df.shape[1] < 9:
        print(f"   Warning: {basename} has too few columns!")
        return [], label

    # Get 8 EEG channels (columns 1-8), transpose so each ROW is one channel
    eeg_data = df.iloc[:, 1:9].values.T   # Shape: (8, num_samples)

    features = []

    for channel_data in eeg_data:
        channel_data = channel_data[~np.isnan(channel_data)]  # Remove NaN values

        if len(channel_data) < SAMPLING_RATE:
            features.extend([0.2] * 5)
            continue

        # THE KEY STEP: Calculate Power Spectral Density using Welch method
        frequencies, power = welch(channel_data, fs=SAMPLING_RATE,
                                    nperseg=min(2048, len(channel_data)))

        # Calculate power in each brainwave band
        band_powers = []
        for band_name, (low, high) in BANDS.items():
            in_band = (frequencies >= low) & (frequencies <= high)
            band_power = np.trapezoid(power[in_band], frequencies[in_band]) if np.any(in_band) else 0.0
            band_powers.append(band_power)

        # Convert to RELATIVE power (percentages adding to 100%)
        total_power = sum(band_powers)
        if total_power == 0:
            total_power = 1.0
        relative_powers = [p / total_power for p in band_powers]
        features.extend(relative_powers)

    return features, label

print("Simple feature extraction function defined!")
print(f"   Output: {NUM_CHANNELS * len(BANDS)} features per file (8 channels x 5 bands)")


## Step B2: Load all files and train the naive model

In [ ]:
print("Loading all training files (naive approach)...\n")

naive_features = []
naive_labels = []

for filename in sorted(os.listdir(DATA_FOLDER)):
    if filename.endswith('.csv') and 'test' not in filename.lower():
        filepath = os.path.join(DATA_FOLDER, filename)
        features, label = extract_features_simple(filepath)
        if features:
            naive_features.append(features)
            naive_labels.append(label)
            print(f"   {filename:35s} -> label: '{label}'")

unique_labels = sorted(set(naive_labels))
print(f"\n{'='*60}")
print(f"NAIVE DATASET SUMMARY")
print(f"{'='*60}")
print(f"   Total samples:    {len(naive_features)}")
print(f"   Unique labels:    {unique_labels}")
print(f"   Number of classes: {len(unique_labels)}")
print()
for label in unique_labels:
    count = naive_labels.count(label)
    print(f"   '{label}': {count} samples")

print(f"\n{'='*60}")
print(f"PROBLEMS WE CAN ALREADY SEE:")
print(f"{'='*60}")
print(f"   1. 'relaxing' and 'relaxation' are DIFFERENT classes!")
print(f"      (But they mean the same thing!)")
print(f"   2. We have {len(unique_labels)} classes but only {len(naive_features)} samples total")
print(f"      (That's only ~{len(naive_features)//max(len(unique_labels),1)} samples per class)")
print(f"   3. Each file gives us only ONE training sample")


In [ ]:
# TRAIN THE NAIVE MODEL

naive_label_to_num = {label: i for i, label in enumerate(unique_labels)}
naive_num_to_label = {i: label for label, i in naive_label_to_num.items()}
y_naive = np.array([naive_label_to_num[l] for l in naive_labels])
X_naive = np.array(naive_features)

X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(
    X_naive, y_naive, test_size=0.3, random_state=42, stratify=y_naive)

naive_model = RandomForestClassifier(n_estimators=100, random_state=42)
naive_model.fit(X_train_n, y_train_n)

y_pred_n = naive_model.predict(X_test_n)
naive_accuracy = accuracy_score(y_test_n, y_pred_n)

print(f"{'='*60}")
print(f"NAIVE MODEL RESULTS")
print(f"{'='*60}")
print(f"\n   Accuracy: {naive_accuracy:.0%}")
if naive_accuracy < 0.5:
    print(f"\n   That's WORSE than flipping a coin!")
    print(f"   (Random guessing = {1/len(unique_labels):.0%})")
print(f"\n   Don't worry - we'll fix this in Part C!")

# Confusion matrix
cm_naive = confusion_matrix(y_test_n, y_pred_n)
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm_naive, cmap='Reds')
ax.set_title(f'Naive Model Confusion Matrix (Accuracy: {naive_accuracy:.0%})', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(unique_labels)))
ax.set_yticks(range(len(unique_labels)))
ax.set_xticklabels(unique_labels, rotation=45, ha='right')
ax.set_yticklabels(unique_labels)
ax.set_xlabel('Predicted', fontsize=12)
ax.set_ylabel('Actual', fontsize=12)
for i in range(len(unique_labels)):
    for j in range(len(unique_labels)):
        ax.text(j, i, str(cm_naive[i, j]), ha='center', va='center',
                fontsize=16, fontweight='bold',
                color='white' if cm_naive[i, j] > cm_naive.max()/2 else 'black')
plt.colorbar(im)
plt.tight_layout()
plt.show()

print("\nThe model is confused because:")
print("   * 'relaxing' and 'relaxation' SHOULD be the same class")
print("   * Too many classes with too few samples per class")
print("   * Each file produces only 1 training sample")
print("\nLet's fix ALL of these in Part C!")


---
# 🅲 PART C: Building the GOOD Model

Now we fix the three problems:

### Fix 1: Merge similar labels into just 2 classes (Binary classification)
- **"relaxed"** = relaxation + relaxing + meditation (calm brain states)
- **"not_relaxed"** = focus + negative + deepthinking (active/stressed brain states)

### Fix 2: Sliding window augmentation gives us 50x more training data!
Instead of 1 sample per file, we slide a 4-second window across each 60-second recording:
```
File: ████████████████████████████████████████████ (60 seconds)
Window 1:  [████]                    -> 1 sample
Window 2:   [████]                   -> 1 sample
Window 3:    [████]                  -> 1 sample
...
Window 56:                    [████] -> 1 sample
Result: 1 file -> ~56 training samples!
```

### Fix 3: Extra "smart" features
We add **alpha/beta ratio** — the single best indicator of relaxation:
- High alpha / low beta -> relaxed
- Low alpha / high beta -> stressed

In [ ]:
# FIX 1: LABEL MAPPING
# Map messy filenames to clean binary labels

LABEL_MAP = {
    'relaxing':     'relaxed',
    'relaxation':   'relaxed',
    'meditation':   'relaxed',
    'negative':     'not_relaxed',
    'focus':        'not_relaxed',
    'deepthinking': 'not_relaxed',
    'stressed':     'not_relaxed',
}

CLASS_NAMES = ['relaxed', 'not_relaxed']

print("Label mapping defined:")
for raw, mapped in sorted(LABEL_MAP.items()):
    emoji = "(calm)" if mapped == "relaxed" else "(active)"
    print(f"   '{raw}' -> '{mapped}' {emoji}")
print(f"\n   From {len(set(LABEL_MAP.keys()))} messy labels -> {len(CLASS_NAMES)} clean classes!")


In [ ]:
# FIX 2 & 3: IMPROVED FEATURE EXTRACTION WITH AUGMENTATION

def extract_band_powers_one_channel(signal_segment):
    # For ONE channel: calculate relative power in each band
    # Returns 5 numbers that add up to ~1.0 (100%)
    if len(signal_segment) < SAMPLING_RATE:
        return [0.2] * len(BANDS)

    nperseg = min(2 * SAMPLING_RATE, len(signal_segment))
    frequencies, power = welch(signal_segment, fs=SAMPLING_RATE, nperseg=nperseg)

    band_powers = []
    for low, high in BANDS.values():
        mask = (frequencies >= low) & (frequencies <= high)
        bp = np.trapezoid(power[mask], frequencies[mask]) if np.any(mask) else 0.0
        band_powers.append(bp)

    total = sum(band_powers)
    if total == 0:
        return [0.2] * len(BANDS)
    return [p / total for p in band_powers]


def extract_features_improved(eeg_window):
    # Extract features from an 8-channel EEG window
    # Returns 44 features: 40 band powers + 4 ratio features
    features = []
    alpha_list = []
    beta_list = []
    theta_list = []

    for ch in range(eeg_window.shape[0]):
        channel_data = eeg_window[ch]
        channel_data = channel_data[~np.isnan(channel_data)]
        bp = extract_band_powers_one_channel(channel_data)
        features.extend(bp)

        # bands order: delta(0), theta(1), alpha(2), beta(3), gamma(4)
        alpha_list.append(bp[2])
        beta_list.append(bp[3])
        theta_list.append(bp[1])

    # FIX 3: SMART RATIO FEATURES (our secret weapon!)
    avg_alpha = np.mean(alpha_list)
    avg_beta = np.mean(beta_list) + 1e-10    # tiny number to prevent divide-by-zero
    avg_theta = np.mean(theta_list)

    features.append(avg_alpha / avg_beta)     # Alpha/Beta ratio = KEY relaxation indicator
    features.append(avg_theta / avg_beta)     # Theta/Beta ratio = meditation indicator
    features.append(avg_alpha)                # Average alpha power
    features.append(avg_beta)                 # Average beta power

    return features


def load_file_with_augmentation(filepath, label, window_sec=4, step_sec=1):
    # Load one CSV and extract MANY training samples using sliding windows
    # A 60-second file -> approximately 56 samples!
    df = pd.read_csv(filepath, header=None, sep='\t')
    if df.shape[1] < 9:
        return [], []

    eeg_data = df.iloc[:, 1:9].values.T
    total_samples = eeg_data.shape[1]
    window_samples = int(window_sec * SAMPLING_RATE)
    step_samples = int(step_sec * SAMPLING_RATE)

    features_list = []
    labels_list = []

    start = 0
    while start + window_samples <= total_samples:
        window = eeg_data[:, start:start + window_samples]
        feats = extract_features_improved(window)
        features_list.append(feats)
        labels_list.append(label)
        start += step_samples

    return features_list, labels_list

print("Improved feature extraction functions defined!")
print(f"   Features per sample: {NUM_CHANNELS * len(BANDS)} band powers + 4 ratios = {NUM_CHANNELS * len(BANDS) + 4} total")
print(f"   Window size: 4 seconds ({4 * SAMPLING_RATE} data points)")
print(f"   Step size: 1 second (lots of overlap = lots of training data!)")


## Step C3: Load all data with the improved approach

In [ ]:
print("Loading and augmenting all training files...\n")

all_features = []
all_labels = []

for filename in sorted(os.listdir(DATA_FOLDER)):
    if not filename.endswith('.csv') or 'test' in filename.lower():
        continue

    raw_label = filename.split('_')[0]
    mapped_label = LABEL_MAP.get(raw_label)

    if mapped_label is None:
        print(f"   Unknown label '{raw_label}' in {filename} - skipping!")
        continue

    filepath = os.path.join(DATA_FOLDER, filename)
    feats, labels = load_file_with_augmentation(filepath, mapped_label)
    all_features.extend(feats)
    all_labels.extend(labels)

    tag = "(calm)" if mapped_label == "relaxed" else "(active)"
    print(f"   {filename:35s} -> {mapped_label:12s} ({len(feats)} windows) {tag}")

X = np.array(all_features)
y = np.array(all_labels)

label_to_num = {'relaxed': 0, 'not_relaxed': 1}
num_to_label = {0: 'relaxed', 1: 'not_relaxed'}
y_encoded = np.array([label_to_num[l] for l in y])

relaxed_count = sum(1 for l in y if l == 'relaxed')
stressed_count = sum(1 for l in y if l == 'not_relaxed')

print(f"\n{'='*60}")
print(f"IMPROVED DATASET SUMMARY")
print(f"{'='*60}")
print(f"   Total samples:     {len(X):,}")
print(f"   Feature dimensions: {X.shape[1]}")
print(f"   Relaxed:         {relaxed_count:,} samples ({relaxed_count/len(y)*100:.1f}%)")
print(f"   Not Relaxed:     {stressed_count:,} samples ({stressed_count/len(y)*100:.1f}%)")
print(f"\n   That's {len(X)//max(len(naive_features),1)}x MORE training data than the naive approach!")


## Step C4: Train the improved model

We use a **Random Forest** with 200 decision trees and **cross-validation** (like taking 5 different practice tests to make sure our model isn't just memorizing).

**What is a Random Forest?**
Imagine asking 200 different "experts" (decision trees) to vote on whether the brain is relaxed or not. Each expert looks at slightly different data. The final answer is whatever the MAJORITY votes for — wisdom of the crowd!

In [ ]:
# TRAIN THE IMPROVED MODEL

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded)

print(f"Training set: {len(X_train):,} samples")
print(f"Test set:     {len(X_test):,} samples\n")

# Model pipeline: normalize features, then Random Forest
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(
        n_estimators=200, max_depth=15, min_samples_leaf=3,
        class_weight='balanced', random_state=42, n_jobs=-1))
])

# Cross-validation
print("Running 5-fold cross-validation...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipeline, X, y_encoded, cv=cv, scoring='accuracy')
print(f"   Fold scores: {[f'{s:.1%}' for s in cv_scores]}")
print(f"   Average:     {cv_scores.mean():.1%} (+/- {cv_scores.std():.1%})")

# Train final model
print(f"\nTraining final model on {len(X_train):,} samples...")
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)

print(f"\n{'='*60}")
print(f"IMPROVED MODEL RESULTS")
print(f"{'='*60}")
print(f"   Test Accuracy: {test_accuracy:.0%}")
print(f"   (Naive model was: {naive_accuracy:.0%})")
print(f"   Improvement: +{test_accuracy - naive_accuracy:.0%}!")
print()
print(classification_report(y_test, y_pred, target_names=CLASS_NAMES))

# Save model
MODEL_PATH = os.path.join(DATA_FOLDER, 'brain_classifier_hackathon.pkl')
joblib.dump({
    'pipeline': pipeline, 'num_to_label': num_to_label,
    'label_to_num': label_to_num, 'class_names': CLASS_NAMES,
    'bands': BANDS, 'sampling_rate': SAMPLING_RATE, 'num_channels': NUM_CHANNELS,
}, MODEL_PATH)
print(f"Model saved to: {MODEL_PATH}")


## Step C5: Visualize — Before vs After

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Bar chart comparison
ax0 = axes[0]
models = ['Naive Model\n(Before)', 'Improved Model\n(After)']
accuracies = [naive_accuracy * 100, test_accuracy * 100]
colors = ['#e74c3c', '#2ecc71']
bars = ax0.bar(models, accuracies, color=colors, width=0.5, edgecolor='black')
ax0.set_ylabel('Accuracy (%)', fontsize=12)
ax0.set_title('Accuracy: Before vs After', fontsize=14, fontweight='bold')
ax0.set_ylim(0, 105)
ax0.axhline(y=50, color='gray', linestyle='--', alpha=0.5, label='Random guessing')
for bar, acc in zip(bars, accuracies):
    ax0.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
             f'{acc:.0f}%', ha='center', fontsize=16, fontweight='bold')
ax0.legend(fontsize=10)

# Improved confusion matrix
ax1 = axes[1]
cm = confusion_matrix(y_test, y_pred)
im = ax1.imshow(cm, cmap='Greens')
ax1.set_title('Improved Confusion Matrix', fontsize=14, fontweight='bold')
ax1.set_xticks([0, 1])
ax1.set_yticks([0, 1])
ax1.set_xticklabels(['Relaxed', 'Not Relaxed'], fontsize=11)
ax1.set_yticklabels(['Relaxed', 'Not Relaxed'], fontsize=11)
ax1.set_xlabel('Predicted', fontsize=12)
ax1.set_ylabel('Actual', fontsize=12)
for i in range(2):
    for j in range(2):
        ax1.text(j, i, str(cm[i, j]), ha='center', va='center',
                fontsize=20, fontweight='bold',
                color='white' if cm[i, j] > cm.max()/2 else 'black')
plt.colorbar(im, ax=ax1)

# Feature importance
ax2 = axes[2]
clf = pipeline.named_steps['clf']
importances = clf.feature_importances_
feat_names = []
for ch in range(NUM_CHANNELS):
    for band in BANDS.keys():
        feat_names.append(f"Ch{ch+1}_{band}")
feat_names.extend(['alpha/beta', 'theta/beta', 'mean_alpha', 'mean_beta'])
top_k = 15
top_idx = np.argsort(importances)[-top_k:]
colors_feat = ['#e74c3c' if 'alpha' in feat_names[i] or 'beta' in feat_names[i] else '#3498db' for i in top_idx]
ax2.barh(range(top_k), importances[top_idx], color=colors_feat)
ax2.set_yticks(range(top_k))
ax2.set_yticklabels([feat_names[i] for i in top_idx], fontsize=9)
ax2.set_title('Top 15 Most Important Features', fontsize=14, fontweight='bold')
ax2.set_xlabel('Importance Score', fontsize=12)

plt.tight_layout()
plt.savefig(os.path.join(DATA_FOLDER, 'model_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

print("Key takeaway: Alpha and Beta features are the most important!")
print("   Alpha waves go UP when relaxed")
print("   Beta waves go UP when stressed/thinking")
print("   The alpha/beta RATIO is the #1 indicator!")


---
# 🅳 PART D: Simulated Live Demo (with Soothing Music!)

Now the fun part! We take a **test recording** (a file the model has NEVER seen) and simulate real-time classification.

When the person is NOT relaxed -> **soothing music plays automatically!**

### About the soothing music
We generate calming ambient tones using healing frequencies (174 Hz, 285 Hz, 396 Hz) with a gentle breathing rhythm. You can also use any .wav/.mp3 from:
- [Pixabay Free Relaxing Music](https://pixabay.com/music/search/relax/) (no attribution required)
- [Mixkit Free Ambient Music](https://mixkit.co/free-stock-music/ambient/)

In [ ]:
# GENERATE SOOTHING MUSIC
import wave

def create_soothing_wav(filename, duration_sec=90, sample_rate=44100):
    t = np.linspace(0, duration_sec, int(sample_rate * duration_sec), endpoint=False)

    # Layer calming frequencies
    signal = np.zeros_like(t)
    signal += 0.15 * np.sin(2 * np.pi * 174 * t)    # Calming tone
    signal += 0.12 * np.sin(2 * np.pi * 285 * t)    # Healing tone
    signal += 0.08 * np.sin(2 * np.pi * 396 * t)    # Releasing tone
    signal += 0.06 * np.sin(2 * np.pi * 176 * t)    # Gentle 2Hz beating
    signal += 0.04 * np.sin(2 * np.pi * 528 * t)    # Soft shimmer

    # Fade in/out
    fade = int(3 * sample_rate)
    signal[:fade] *= np.linspace(0, 1, fade)
    signal[-fade:] *= np.linspace(1, 0, fade)

    # Breathing effect
    breathing = 0.8 + 0.2 * np.sin(2 * np.pi * 0.15 * t)
    signal *= breathing

    signal = signal / np.max(np.abs(signal)) * 0.7
    signal_int = (signal * 32767).astype(np.int16)

    with wave.open(filename, 'w') as f:
        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(sample_rate)
        f.writeframes(signal_int.tobytes())
    print(f"Created: {filename} ({duration_sec}s of calming ambient sound)")

MUSIC_FILE = os.path.join(DATA_FOLDER, 'soothing_music.wav')
create_soothing_wav(MUSIC_FILE)

# Set up audio playback
music_available = False
try:
    import pygame
    pygame.mixer.init()
    if os.path.exists(MUSIC_FILE):
        pygame.mixer.music.load(MUSIC_FILE)
        music_available = True
        print("Audio playback ready!")
    else:
        print("Music file not found (visual demo only)")
except Exception as e:
    print(f"Audio not available: {e}")
    print("   (Normal in Colab - music works on your laptop!)")

def play_soothing_music():
    if music_available and not pygame.mixer.music.get_busy():
        pygame.mixer.music.play(-1)

def stop_soothing_music():
    if music_available and pygame.mixer.music.get_busy():
        pygame.mixer.music.fadeout(1000)


In [ ]:
# SIMULATED LIVE DEMO FUNCTION

def simulated_live_demo(filename, window_sec=4, step_sec=2):
    saved = joblib.load(MODEL_PATH)
    demo_pipeline = saved['pipeline']
    demo_num_to_label = saved['num_to_label']

    print(f"\n{'='*60}")
    print(f"SIMULATED LIVE BRAIN STATE CLASSIFICATION")
    print(f"{'='*60}")
    print(f"  File:     {os.path.basename(filename)}")
    print(f"  Window:   {window_sec} seconds")
    print(f"  Update:   every {step_sec} seconds")
    print(f"  Music:    {'Ready' if music_available else 'Visual only'}")
    print(f"{'='*60}\n")

    df = pd.read_csv(filename, header=None, sep='\t')
    if df.shape[1] < 9:
        print("File has too few columns!")
        return

    eeg_data = df.iloc[:, 1:9].values.T
    window_samples = int(window_sec * SAMPLING_RATE)
    step_samples = int(step_sec * SAMPLING_RATE)

    times, pred_labels, confidences = [], [], []
    music_playing = False
    start = 0

    while start + window_samples <= eeg_data.shape[1]:
        end = start + window_samples
        window = eeg_data[:, start:end]

        features = extract_features_improved(window)
        feature_vec = np.array(features).reshape(1, -1)

        pred_num = demo_pipeline.predict(feature_vec)[0]
        pred_proba = demo_pipeline.predict_proba(feature_vec)[0]
        confidence = max(pred_proba)
        label = demo_num_to_label[pred_num]

        mid_time = (start + end) / 2 / SAMPLING_RATE
        times.append(mid_time)
        pred_labels.append(label)
        confidences.append(confidence)

        emoji = "Relaxed    " if label == "relaxed" else "NOT RELAXED"
        conf_bar = "#" * int(confidence * 20) + "." * (20 - int(confidence * 20))
        print(f"  {mid_time:6.1f}s  {emoji}  [{conf_bar}] {confidence:.0%}")

        if label == 'not_relaxed' and not music_playing:
            play_soothing_music()
            music_playing = True
            print(f"           >>> Playing soothing music...")
        elif label == 'relaxed' and music_playing:
            stop_soothing_music()
            music_playing = False
            print(f"           >>> Music stopped - brain is relaxed!")
        start += step_samples

    stop_soothing_music()

    # Plot results
    CLASS_COLORS = {'relaxed': '#2ecc71', 'not_relaxed': '#e74c3c'}
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), height_ratios=[3, 1])

    for i, (t, label) in enumerate(zip(times, pred_labels)):
        color = CLASS_COLORS[label]
        ax1.axvspan(t - step_sec/2, t + step_sec/2, alpha=0.4, color=color)
        ax1.plot(t, 1 if label == 'relaxed' else 0, 'o', color=color, markersize=10, zorder=5)

    ax1.set_yticks([0, 1])
    ax1.set_yticklabels(['Not Relaxed\n(stressed/focused)', 'Relaxed\n(calm/meditative)'], fontsize=13)
    ax1.set_xlabel('Time (seconds)', fontsize=12)
    ax1.set_title('Brain State Over Time (with Auto-Soothing Music)', fontsize=16, fontweight='bold')
    ax1.grid(True, axis='x', alpha=0.3)
    patches = [mpatches.Patch(color='#2ecc71', label='Relaxed', alpha=0.6),
               mpatches.Patch(color='#e74c3c', label='Not Relaxed (music plays)', alpha=0.6)]
    ax1.legend(handles=patches, loc='upper right', fontsize=12)

    ax2.fill_between(times, confidences, alpha=0.3, color='#3498db')
    ax2.plot(times, confidences, color='#2980b9', linewidth=2)
    ax2.set_ylabel('Model Confidence', fontsize=12)
    ax2.set_xlabel('Time (seconds)', fontsize=12)
    ax2.set_ylim(0.4, 1.05)
    ax2.axhline(y=0.7, color='orange', linestyle='--', alpha=0.5, label='70% threshold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(DATA_FOLDER, 'live_demo_result.png'), dpi=150, bbox_inches='tight')
    plt.show()

    relaxed_pct = sum(1 for l in pred_labels if l == 'relaxed') / len(pred_labels) * 100
    print(f"\n{'='*60}")
    print(f"SESSION SUMMARY")
    print(f"{'='*60}")
    print(f"  Total time:         {times[-1]:.0f} seconds")
    print(f"  Relaxed:            {relaxed_pct:.0f}% of session")
    print(f"  Not relaxed:        {100-relaxed_pct:.0f}% of session")
    print(f"  Average confidence: {np.mean(confidences):.0%}")
    print(f"{'='*60}")

print("Demo function ready!")


In [ ]:
# RUN THE SIMULATED LIVE DEMO!

test_file = os.path.join(DATA_FOLDER, 'test_subject.csv')

if os.path.exists(test_file):
    simulated_live_demo(test_file, window_sec=4, step_sec=2)
else:
    print(f"Test file not found: test_subject.csv")
    print(f"\nRunning on a training file as demo instead:")
    for f in sorted(os.listdir(DATA_FOLDER)):
        if f.endswith('.csv') and 'test' not in f.lower():
            print(f"   Using: {f}")
            simulated_live_demo(os.path.join(DATA_FOLDER, f), window_sec=4, step_sec=2)
            break


---
# 🅴 PART E: Real-Time Live Demo with OpenBCI Headset

**The showstopper!** Connect your headset and classify brain state in real-time.

### Setup checklist:
1. OpenBCI headset is powered ON
2. Bluetooth dongle is plugged into your laptop
3. OpenBCI GUI is **CLOSED** (BrainFlow needs exclusive access)
4. You know your serial port (see below)

### Finding your serial port:
- **Windows**: Device Manager -> Ports -> look for "Silicon Labs" -> e.g. `COM5`
- **Mac**: Terminal -> `ls /dev/tty.*` -> look for `tty.usbserial-XXXXX`
- **Linux**: Terminal -> `ls /dev/ttyUSB*` -> usually `/dev/ttyUSB0`

### How it works:
```
OpenBCI Headset (on head)  ->  Bluetooth  ->  USB Dongle  ->  BrainFlow  ->  Our Model  ->  Music!
```

In [ ]:
# CHANGE THIS TO YOUR PORT!
SERIAL_PORT = 'COM5'    # Windows: 'COM5', Mac: '/dev/tty.usbserial-XXXXX', Linux: '/dev/ttyUSB0'
print(f"Serial port: {SERIAL_PORT}")
print("(Change in the cell above if needed!)")


In [ ]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes

def live_realtime_demo(port=SERIAL_PORT, window_sec=4, update_sec=1, duration_sec=120):
    print(f"\n{'='*60}")
    print(f"REAL-TIME BRAIN STATE CLASSIFICATION")
    print(f"{'='*60}")
    print(f"  Port: {port} | Window: {window_sec}s | Duration: {duration_sec}s")
    print(f"{'='*60}")

    saved = joblib.load(MODEL_PATH)
    rt_pipeline = saved['pipeline']
    rt_num_to_label = saved['num_to_label']

    params = BrainFlowInputParams()
    params.serial_port = port
    board = BoardShim(BoardIds.CYTON_BOARD.value, params)
    sampling_rate = BoardShim.get_sampling_rate(BoardIds.CYTON_BOARD.value)
    eeg_channels = BoardShim.get_eeg_channels(BoardIds.CYTON_BOARD.value)
    window_samples = int(window_sec * sampling_rate)

    print(f"\nConnecting to OpenBCI Cyton...")

    try:
        board.prepare_session()
        board.start_stream()
        print("Connected! Streaming started!")
        print(f"\nBuffering data ({window_sec + 1} seconds)...\n")
        time.sleep(window_sec + 1)

        music_playing = False
        start_time = time.time()
        print("Classification started! (Ctrl+C to stop)\n")

        while (time.time() - start_time) < duration_sec:
            data = board.get_current_board_data(window_samples)
            if data.shape[1] < window_samples:
                time.sleep(0.5)
                continue

            eeg_window = data[eeg_channels, :]
            for ch in range(eeg_window.shape[0]):
                DataFilter.perform_bandpass(eeg_window[ch], sampling_rate, 1.0, 45.0, 4, FilterTypes.BUTTERWORTH.value, 0)
                DataFilter.perform_bandstop(eeg_window[ch], sampling_rate, 58.0, 62.0, 4, FilterTypes.BUTTERWORTH.value, 0)

            features = extract_features_improved(eeg_window)
            feature_vec = np.array(features).reshape(1, -1)
            pred_num = rt_pipeline.predict(feature_vec)[0]
            pred_proba = rt_pipeline.predict_proba(feature_vec)[0]
            confidence = max(pred_proba)
            label = rt_num_to_label[pred_num]

            elapsed = time.time() - start_time
            tag = "Relaxed    " if label == "relaxed" else "NOT RELAXED"
            conf_bar = "#" * int(confidence * 20) + "." * (20 - int(confidence * 20))
            print(f"  [{time.strftime('%H:%M:%S')}] {tag} [{conf_bar}] {confidence:.0%}  ({elapsed:.0f}s/{duration_sec}s)")

            if label == 'not_relaxed' and not music_playing:
                play_soothing_music()
                music_playing = True
                print("           >>> Soothing music started...")
            elif label == 'relaxed' and music_playing:
                stop_soothing_music()
                music_playing = False
                print("           >>> Music stopped - relaxed!")
            time.sleep(update_sec)

    except KeyboardInterrupt:
        print("\n\nDemo stopped by user.")
    except Exception as e:
        print(f"\nConnection error: {e}")
        print(f"   1. Is port '{port}' correct?")
        print(f"   2. Is headset powered ON?")
        print(f"   3. Is Bluetooth dongle plugged in?")
        print(f"   4. Is OpenBCI GUI CLOSED?")
    finally:
        stop_soothing_music()
        try:
            board.stop_stream()
            board.release_session()
        except:
            pass
        print("\nDisconnected safely. Demo complete!")

print("Real-time demo function ready!")


In [ ]:
# RUN THE REAL-TIME DEMO!
# Uncomment the line below when your headset is connected:

# live_realtime_demo(port=SERIAL_PORT, window_sec=4, update_sec=1, duration_sec=120)

print("Uncomment the line above when your OpenBCI headset is connected!")
print()
print("Demo script for the judges:")
print("   1. Have a volunteer put on the headset")
print("   2. Run the cell above")
print("   3. Ask them to CLOSE THEIR EYES and breathe slowly (30 sec)")
print("      -> Model should say 'Relaxed'")
print("   4. Ask them to COUNT BACKWARDS from 1000 by 7s (1000, 993, 986...)")
print("      -> Model should say 'NOT RELAXED'")
print("      -> Soothing music starts playing!")
print("   5. Ask them to RELAX again (close eyes, breathe)")
print("      -> Music fades out as they calm down!")


---
# 🅵 PART F: Preparing for Judge Questions

Here are the most likely questions judges will ask, with answers you can give confidently.

---

### Q1: "How does your model work?"

**Answer:** "We place 8 electrodes on the scalp using an OpenBCI headset. These measure tiny electrical signals called brainwaves. We break these signals into 5 frequency bands using a technique called Power Spectral Density. We calculate what percentage of total brain power is in each band. When someone is relaxed, alpha waves (8-13 Hz) are strong. When stressed, beta waves (13-30 Hz) dominate. We feed these 44 numbers into a Random Forest classifier (200 decision trees voting together) and it predicts relaxed vs. not relaxed."

---

### Q2: "Why was your first model so bad?"

**Answer:** "Three reasons! First, inconsistent labels: 'relaxing' and 'relaxation' were treated as different classes. Second, 5 classes with only ~19 files was too few for ML. Third, each 60-second recording only gave us one training sample. We fixed all three: merged labels into 2 classes, and used sliding-window augmentation to get 50+ samples from each recording."

---

### Q3: "What is a Random Forest?"

**Answer:** "Imagine asking 200 people to vote on whether brain data shows relaxation or stress. Each person (decision tree) looks at slightly different features and might make small mistakes. But the majority vote of all 200 is usually right! That's a Random Forest — wisdom of the crowd for machine learning."

---

### Q4: "What are alpha and beta waves?"

**Answer:** "Alpha waves oscillate at 8-13 Hz and are strongest when you're calm with eyes closed. Beta waves are faster at 13-30 Hz and increase when thinking hard or stressed. The ratio between alpha and beta power is the single best indicator of relaxation — and it's our model's most important feature."

---

### Q5: "What is cross-validation?"

**Answer:** "It's like taking 5 practice tests instead of one. We split data into 5 parts, train on 4 and test on 1, rotating which part is the test. Every sample gets tested exactly once, giving us a reliable accuracy estimate instead of just one lucky or unlucky split."

---

### Q6: "How does the music feature work?"

**Answer:** "Every second, the model predicts relaxed or not. If not relaxed, it starts playing calming ambient music with healing frequencies (174, 285, 396 Hz) and a breathing rhythm. Once it detects relaxation, the music fades out. It's a real-time biofeedback loop — the brain responds to the music, and the model detects the change!"

---

### Q7: "What could you improve?"

**Answer:** "More training data (30+ minutes per class), trying deep learning models like convolutional neural networks, adding more brain states (drowsy, focused, creative), building a real-time visual dashboard, and letting users pick their own music."

---

### Q8: "Is this clinically accurate?"

**Answer:** "This is a prototype, not a medical device. Clinical EEG uses many more electrodes and hospital-grade equipment. But the underlying science is real — the relationship between alpha/beta waves and mental state is well-established in neuroscience. Consumer EEG devices are being used for meditation apps, ADHD treatment, and sleep monitoring."

---

### Q9: "How much data did you need?"

**Answer:** "We started with only 19 recordings (about 19 minutes total). Using sliding-window augmentation with a 4-second window and 1-second step, we turned those into over 1,000 training samples. More data always helps — we'd recommend 5+ minutes per recording and 10+ recordings per class."

---

### Q10: "What's the Welch method?"

**Answer:** "It's a way to find which frequencies are strongest in a signal. It breaks the signal into small overlapping chunks, uses the Fourier Transform on each to find frequencies, then averages everything. The averaging reduces noise and gives a cleaner picture than a single Fourier Transform."


---
## You're Ready for the Hackathon!

### Demo flow for judges (10 minutes):
1. **2 min** — Explain the concept (brain -> headset -> classifier -> music)
2. **1 min** — Show the naive model failing (Part B)
3. **2 min** — Show how you fixed it (Part C)
4. **3 min** — Live demo with a volunteer (Part E)
5. **2 min** — Answer questions (Part F)

### Good luck! 🧠🏆
